In [1]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

dataset = load_dataset('ag_news')
train_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
validation_test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(1000))])


display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default
Reusing dataset ag_news (C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 66.78it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-dd0ff9596fea92b0.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-12f3c4e4bf422cce.arrow


Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

,text,label
0,Bangladesh paralysed by strikes Opposition act...,0
1,Desiring Stability Redskins coach Joe Gibbs ex...,1
2,Will Putin #39;s Power Play Make Russia Safer?...,0
3,U2 pitches for Apple New iTunes ads airing dur...,3
4,S African TV in beheading blunder Public broad...,0
...,...,...
4995,Nigerian Protection Force Leaves for Darfur (R...,0
4996,"Lawrence Dallaglio factfile 1995: In October, ...",1
4997,African leaders reject foreign intervention in...,0
4998,Ex-Troops Fill Haiti's Security Vacuum Nation ...,0


In [2]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [3]:
#creating synonyms list using wordnet
def synonyms_list_creation(target_word):
    synonyms_list = []
    try:
        for syn in wordnet.synsets(target_word):
            for lm in syn.lemmas():
                synonyms_list.append(lm.name())
        #print (set(synonyms_list))
        synonyms_list = list(set(synonyms_list))
        #transform _ to space
        synonyms_list = [s.replace('_', ' ') for s in synonyms_list]


        synonyms_list
        synonyms_list = [s for s in synonyms_list if ' ' not in s]
    
    except Exception as e:
        pass
    
    return synonyms_list

In [5]:
from textattack.transformations import WordSwap
class Same_POS_Synonyms_WordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        synonyms_list = synonyms_list_creation(word)
        
        return synonyms_list

In [6]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore

from textattack.augmentation import Augmenter


transformation = Same_POS_Synonyms_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(), 
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=False, compare_against_original=True, language_nltk='eng', language_stanza='en'),
                BERTScore(min_bert_score = 0.7, model_name='bert-base-uncased', num_layers=None, score_type='f1', compare_against_original=True)]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

In [7]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):
    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)



time_end = time.perf_counter()
tim = time_end - time_sta

In [8]:


print(tim/3600)




8.348431926305555


In [9]:
display(results_list)

[['Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 citizenry exit in explosions at a political rally.'],
 ['Desiring Stability Redskins jitney Joe Gibbs ask few major personnel changes in the offseason and wants to instill a culture of stability in Washington.'],
 ['Will Putin #39;s mightiness Play Make Russia Safer? Outwardly, Russia has not changed since the barrage of terrorist attacks that culminate in the schooling massacre in Beslan on Sept.'],
 ['U2 pitches for Apple raw iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish rockers.'],
 ['S African video in beheading botch Public broadcaster SABC apologises after news bulletin shows footage of American beheaded in Iraq.'],
 ['A Cosmic Storm: When Galaxy Clusters Collide Astronomers have ascertain what they are shout the perfect cosmic storm, a galaxy cluster pile-up so powerful its vigor output is second only to the

In [10]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.'],
 ['Desiring Stability Redskins coach Joe Gibbs expects few major personnel changes in the offseason and wants to instill a culture of stability in Washington.'],
 ['Will Putin #39;s Power Play Make Russia Safer? Outwardly, Russia has not changed since the barrage of terrorist attacks that culminated in the school massacre in Beslan on Sept.'],
 ['U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish rockers.'],
 ['S African TV in beheading blunder Public broadcaster SABC apologises after news bulletin shows footage of American beheaded in Iraq.'],
 ['A Cosmic Storm: When Galaxy Clusters Collide Astronomers have found what they are calling the perfect cosmic storm, a galaxy cluster pile-up so powerful its energy output is second only to the Big Bang

In [11]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,Bangladesh paralysed by strikes Opposition act...,Bangladesh paralysed by strikes Opposition act...
1,Desiring Stability Redskins coach Joe Gibbs ex...,Desiring Stability Redskins jitney Joe Gibbs a...
2,Will Putin #39;s Power Play Make Russia Safer?...,Will Putin #39;s mightiness Play Make Russia S...
3,U2 pitches for Apple New iTunes ads airing dur...,U2 pitches for Apple raw iTunes ads airing dur...
4,S African TV in beheading blunder Public broad...,S African video in beheading botch Public broa...
...,...,...
4995,Nigerian Protection Force Leaves for Darfur (R...,Nigerian Protection Force Leaves for Darfur (R...
4996,"Lawrence Dallaglio factfile 1995: In October, ...","Lawrence Dallaglio factfile 1995: In October, ..."
4997,African leaders reject foreign intervention in...,African leaders disdain foreign intervention i...
4998,Ex-Troops Fill Haiti's Security Vacuum Nation ...,Ex-Troops Fill Haiti's Security Vacuum Nation ...


In [12]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,Bangladesh paralysed by strikes Opposition act...,Bangladesh paralysed by strikes Opposition act...
1,Desiring Stability Redskins coach Joe Gibbs ex...,Desiring Stability Redskins jitney Joe Gibbs a...
2,Will Putin #39;s Power Play Make Russia Safer?...,Will Putin #39;s mightiness Play Make Russia S...
3,U2 pitches for Apple New iTunes ads airing dur...,U2 pitches for Apple raw iTunes ads airing dur...
4,S African TV in beheading blunder Public broad...,S African video in beheading botch Public broa...
...,...,...
4995,Nigerian Protection Force Leaves for Darfur (R...,Nigerian Protection Force Leaves for Darfur (R...
4996,"Lawrence Dallaglio factfile 1995: In October, ...","Lawrence Dallaglio factfile 1995: In October, ..."
4997,African leaders reject foreign intervention in...,African leaders disdain foreign intervention i...
4998,Ex-Troops Fill Haiti's Security Vacuum Nation ...,Ex-Troops Fill Haiti's Security Vacuum Nation ...


In [13]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\ag_news_dataset\augmented_dataset\synonyms_bert_score=0.7_pct=0.1_transformations_per_example=1\dataset.csv')